In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os

In [2]:
os.listdir('/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos')

['violence video cleaned', 'Non-Violence Videos', 'Weapon Violence']

In [3]:
df = pd.DataFrame()
df['filename'] = os.listdir(r'/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/Non-Violence Videos')
df['path'] = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/Non-Violence Videos/'
df['category'] = 0
df2 = pd.DataFrame()
df2['filename'] = os.listdir(r'/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/violence video cleaned')
df2['path'] = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/violence video cleaned/'
df2['category'] = 1
df3 = pd.DataFrame()
df3['filename'] = os.listdir(r'/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/Weapon Violence')
df3['path'] = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/Weapon Violence/'
df3['category'] = 2

df = pd.concat([df, df2, df3])
df = df.sample(frac = 1)
df.head()
df.to_csv("fights2.csv", index=False)

In [4]:
df.head()

,filename,path,category
217,nv69.mp4,/kaggle/input/smartcity-cctv-violence-detectio...,0
115,WV94.mp4,/kaggle/input/smartcity-cctv-violence-detectio...,2
206,nv87.mp4,/kaggle/input/smartcity-cctv-violence-detectio...,0
84,WV36.mp4,/kaggle/input/smartcity-cctv-violence-detectio...,2
46,WV12.mp4,/kaggle/input/smartcity-cctv-violence-detectio...,2


In [5]:
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, LSTM, Dense, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds
import joblib

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
csv_file_path = 'fights2.csv'
data = pd.read_csv(csv_file_path)
data = data.drop(data[data['filename'] == 'V5.mp4'].index)
data.shape


(481, 3)

In [7]:
FRAME_WIDTH = 128
FRAME_HEIGHT = 128
NUM_CLASSES = 3
NUM_FRAMES = 30
BATCH_SIZE = 10

In [8]:
def load_and_preprocess_video(file_path):
    cap = cv2.VideoCapture(file_path)
    frames = []
    # NUM_FRAMES = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    for _ in range(NUM_FRAMES):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
        frames.append(frame)

    cap.release()

    # Pad or truncate frames to ensure a fixed number
    while len(frames) < NUM_FRAMES:
        frames.append(np.zeros((FRAME_HEIGHT, FRAME_WIDTH, 3), dtype=np.uint8))

    return np.array(frames)

In [9]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

# Define a custom data generator function
def data_generator(data, batch_size):
    num_samples = data.shape[0]
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]
            batch_data = data.iloc[batch_indices]
            batch_data.to_csv('bdata.csv')
            batch_X = []
            batch_y = []
            
            for _, row in batch_data.iterrows():
                video_file = row['filename']
                path  = row['path']
                path += video_file
                label = row['category']
                frames = load_and_preprocess_video(path)
                frames = list(map(lambda x :x/255, frames))

                batch_X.append(frames)
                batch_y.append(label)
                del frames
            
            batch_X = np.array(batch_X)
            batch_y = np.array(batch_y)
            yield batch_X, batch_y

In [ ]:
# # Build the 3D CNN model
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, 3)))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Conv3D(128, (3, 3, 3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# shape = model.layers[-1].output_shape
# model.add(Reshape((shape[-1],shape[1]*shape[2]*shape[3])))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64))
# model.add(Dropout(.5))

# model.add((Flatten()))

# # # FC layers group

# model.add(Dense(64, activation='relu'))
# model.add(Dropout(.5))
# model.add(Dense(NUM_CLASSES, activation='softmax'))

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# # model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))
# train_data_generator = data_generator(data_train, BATCH_SIZE)
# validation_data = data_generator(data_test, BATCH_SIZE)
# history = model.fit(
#     train_data_generator,
#     steps_per_epoch=len(data_train) // BATCH_SIZE,
#     epochs=10,
#     validation_data=validation_data,
#     validation_steps=len(data_test) // BATCH_SIZE
# )
# test_loss, test_accuracy = model.evaluate(validation_data, steps=len(data_test) // BATCH_SIZE)
# print(f'Test accuracy: {test_accuracy * 100:.2f}%')
# file1 = open('accuracy.txt', 'w')
# file1.write(f'Test accuracy: {test_accuracy * 100:.2f}%')
# file1.close()

# model.save('fights2vgg.keras')
# f.close()


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

# Define a directory to save the model checkpoints
checkpoint_dir = 'model_checkpoints/'

# Ensure the directory exists
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_weights_vgg.h5'),  # Save with epoch number
    save_best_only=False,  # Save model weights after each epoch
    save_weights_only=True,  # Save only the weights, not the entire model
    verbose=1  # Display progress during saving
)

early_stopping = EarlyStopping(patience = 2, monitor='val_loss',
                                 mode='min', restore_best_weights=True, 
                                 verbose = 1, min_delta = .00075)
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if ((logs.get('accuracy')>=0.999)):
            print("\nLimits Reached cancelling training!")
            self.model.stop_training = True
end_callback = myCallback()


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Reshape
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(FRAME_HEIGHT, FRAME_WIDTH, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a custom video classification model
model = Sequential([
    TimeDistributed(base_model, input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, 3)),
    Flatten(),
    Reshape((NUM_FRAMES, -1)),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create the data generator instances
train_data_generator = data_generator(data_train, BATCH_SIZE)
validation_data = data_generator(data_test, BATCH_SIZE)
# Train the model using fit_generator
history = model.fit(
    train_data_generator,
    steps_per_epoch=len(data_train) // BATCH_SIZE,
    epochs=20,
    validation_data=validation_data,
    validation_steps=len(data_test) // BATCH_SIZE,
    callbacks=[checkpoint_callback, end_callback] 
)
test_loss, test_accuracy = model.evaluate(validation_data, steps=len(data_test) // BATCH_SIZE)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')
file1 = open('accuracy.txt', 'w')
file1.write(f'Test accuracy: {test_accuracy * 100:.2f}%')
file1.close()
model.save('fights2vgg.keras')

In [ ]:
import keras
model = keras.models.load_model('fights2vgg2.keras')

train_data_generator = data_generator(data_train, BATCH_SIZE)
validation_data = data_generator(data_test, BATCH_SIZE)



test_loss, test_accuracy = model.evaluate(validation_data, steps=len(data_test) // BATCH_SIZE)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')
file1 = open('accuracy.txt', 'w')
file1.write(f'Test accuracy: {test_accuracy * 100:.2f}%')
file1.close()
model.save('fights2vgg2.keras')

In [ ]:
import keras
model = keras.models.load_model('fights2vgg2.keras')
frames = load_and_preprocess_video(r'/kaggle/input/cctv-fights-dataset/dataset/CCTV_DATA/training/fight_0012.mpeg')
frames = list(map(lambda x :x/255, frames))
frames = np.expand_dims(frames, axis=0)
p = model.predict(frames)
print(p)

## Only 2 categories

In [10]:
df = pd.DataFrame()
df['filename'] = os.listdir(r'/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/Non-Violence Videos')
df['path'] = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/Non-Violence Videos/'
df['category'] = 0
df2 = pd.DataFrame()
df2['filename'] = os.listdir(r'/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/violence video cleaned')
df2['path'] = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/videos/violence video cleaned/'
df2['category'] = 1
df3 = pd.DataFrame()
df3['filename'] = os.listdir(r'/kaggle/input/cctv-fights-dataset/dataset/CCTV_DATA/training')
df3['path'] = '/kaggle/input/cctv-fights-dataset/dataset/CCTV_DATA/training/'
df3['category'] = 1

data = pd.concat([df2,df, df3])

In [11]:
for idx, row in data.iterrows():
    video_file = row['filename']
    path = row['path']+video_file
    label = row['category']
    frames = load_and_preprocess_video(path)
    frames = list(map(lambda x :x/255, frames))
    if np.sum(frames) == 0:
        data = data.drop(idx)

[av1 @ 0x5729f14b8840] Your platform doesn't suppport hardware accelerated AV1 decoding.
[av1 @ 0x5729f14b8840] Failed to get pixel format.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Header.
[av1 @ 0x5729f14b8840] Missing Sequence Head

In [15]:
NUM_CLASSES = 2

In [16]:
def load_and_preprocess_video(file_path):
    cap = cv2.VideoCapture(file_path)
    frames = []
    # NUM_FRAMES = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    for _ in range(NUM_FRAMES):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
        frames.append(frame)

    cap.release()

    # Pad or truncate frames to ensure a fixed number
    while len(frames) < NUM_FRAMES:
        frames.append(np.zeros((FRAME_HEIGHT, FRAME_WIDTH, 3), dtype=np.uint8))

    return np.array(frames)

In [17]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

# Define a custom data generator function
def data_generator(data, batch_size):
    num_samples = data.shape[0]
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]
            batch_data = data.iloc[batch_indices]
            batch_data.to_csv('bdata.csv')
            batch_X = []
            batch_y = []
            
            for _, row in batch_data.iterrows():
                video_file = row['filename']
                path  = row['path']
                path += video_file
                label = row['category']
                frames = load_and_preprocess_video(path)
                frames = list(map(lambda x :x/255, frames))

                batch_X.append(frames)
                batch_y.append(label)
                del frames
            
            batch_X = np.array(batch_X)
            batch_y = np.array(batch_y)
            yield batch_X, batch_y

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

# Define a directory to save the model checkpoints
checkpoint_dir = 'model_checkpoints/'

# Ensure the directory exists
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_weights_vgg_2cat.h5'),  # Save with epoch number
    save_best_only=False,  # Save model weights after each epoch
    save_weights_only=True,  # Save only the weights, not the entire model
    verbose=1  # Display progress during saving
)

early_stopping = EarlyStopping(patience = 2, monitor='val_loss',
                                 mode='min', restore_best_weights=True, 
                                 verbose = 1, min_delta = .000075)
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if ((logs.get('accuracy')>=0.999)):
            print("\nLimits Reached cancelling training!")
            self.model.stop_training = True
end_callback = myCallback()


In [19]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Flatten, Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Flatten, Reshape
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(FRAME_HEIGHT, FRAME_WIDTH, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a custom video classification model
model = Sequential([
    TimeDistributed(base_model, input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, 3)),
    Flatten(),
    Reshape((NUM_FRAMES, -1)),
    LSTM(30, return_sequences=True),
    LSTM(30),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create the data generator instances
train_data_generator = data_generator(data_train, BATCH_SIZE)
validation_data = data_generator(data_test, BATCH_SIZE)
# Train the model using fit_generator
history = model.fit(
    train_data_generator,
    steps_per_epoch=len(data_train) // BATCH_SIZE,
    epochs=20,
    validation_data=validation_data,
    validation_steps=len(data_test) // BATCH_SIZE,
    callbacks=[checkpoint_callback, end_callback] 
)
test_loss, test_accuracy = model.evaluate(validation_data, steps=len(data_test) // BATCH_SIZE)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')
file1 = open('accuracy.txt', 'w')
file1.write(f'Test accuracy: {test_accuracy * 100:.2f}%')
file1.close()
model.save('fights2vgg2cat2.keras')

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/20
39/39 [==============================] - ETA: 0s - loss: 0.6136 - accuracy: 0.6282
Epoch 1: saving model to model_checkpoints/model_weights_vgg_2cat.h5
39/39 [==============================] - 61s 1s/step - loss: 0.6136 - accuracy: 0.6282 - val_loss: 0.5867 - val_accuracy: 0.6889
Epoch 2/20
39/39 [==============================] - ETA: 0s - loss: 0.4545 - accuracy: 0.8068
Epoch 2: saving model to model_checkpoints/model_weights_vgg_2cat.h5
39/39 [==============================] - 43s 1s/step - loss: 0.4545 - accuracy: 0.8068 - val_loss: 0.5324 - val_accuracy: 0.7667
Epoch 3/20
39/39 [==============================] - ETA: 0s - loss: 0.3486 - accuracy: 0.8590
Epoch 3: saving model to model_checkpoints/model_weights_vgg_2cat.h5
39/39 [==============================] - 42s 1s/step - loss: 0.3486 - accuracy: 0.8590 - val_loss: 0.3834 - val_accuracy: 0.8556
Epoch 4/20
39/39 [==============================] - ETA: 0s

In [27]:
import keras
model = keras.models.load_model('fights2vgg2cat2.keras')
frames = load_and_preprocess_video(r'/kaggle/input/cctv-fights-dataset/dataset/CCTV_DATA/testing/fight_0015.mpeg')
frames = list(map(lambda x :x/255, frames))
frames = np.expand_dims(frames, axis=0)
p = model.predict(frames)
print(p)

1/1 [==============================] - 1s 809ms/step
[[0.00138326 0.9986167 ]]
